In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
print(os.getenv("OPENAI_API_KEY"))

In [ ]:
import openai

client = openai.OpenAI()

messages = []

In [ ]:
def get_weather(city):
    return '30도.'

FUNCTION_MAP = {
    'get_weather': get_weather
}

In [ ]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",
                    },
                },
                "required": ["city"],
            },
        },
	},
]

In [ ]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS
    )
    print(response)
    message = response.choices[0].message.content
    messages.append({
        "role": "assistant",
        "content": message
    })
    print(f"AI: {message}")

In [26]:
while True:
    message = input("Send a message to the LLM...")
    
    if message == "quit" or message == "q":
        break
    
    messages.append({
        "role": "user",
        "content": message
    })
    print(f"User: {message}")
    
    call_ai()

User: What is the weather in Korea?
ChatCompletion(id='chatcmpl-CEWH7RhFvs8QVJ2fiKij40tqhAGuL', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_8fUpKGk38SGkva7H8mlf4pA7', function=Function(arguments='{"city":"Seoul, Korea"}', name='get_weather'), type='function')]))], created=1757576693, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_8bda4d3a2c', usage=CompletionUsage(completion_tokens=17, prompt_tokens=66, total_tokens=83, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
AI: None
